# Descriptions

# For each county, get list of BAs who serve the county 

In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict


In [4]:
dir_files = '../test/data/'

county_utility_file = 'Service_Territory_2016.xlsx'  #get utility-county mapping
ba_utility_file = 'Sales_Ult_Cust_2016.xlsx'  #Get utility-BA mapping
# ba_list_file = 'Balancing_Authority_2016.xlsx'  #list of BAs by state

county_utility =  pd.read_excel(io = dir_files + '/' + county_utility_file, 
                                   dtype = {'Utility Number': str, 'County': str, 'State': str}
                               )
ba_utility = pd.read_excel(io = dir_files + '/' + ba_utility_file, 
                          usecols = 'B,G,I,W', skiprows = [0,1], 
                          dtype = {'Utility Number': str, 'BA_CODE': str, 'State': str},
                          sheet_name = 'States'
                          )
ba_county = pd.merge(county_utility, ba_utility, how='outer', on ='Utility Number')

In [21]:
county_utility.head()

,Data Year,Utility Number,Utility Name,State,County
0,2016,34,City of Abbeville - (SC),SC,Abbeville
1,2016,55,City of Aberdeen - (MS),MS,Monroe
2,2016,59,City of Abbeville - (LA),LA,Vermilion
3,2016,84,A & N Electric Coop,MD,Somerset
4,2016,84,A & N Electric Coop,VA,Accomack


In [22]:
ba_utility.head()

,Utility Number,State,BA_CODE,Megawatthours.4
0,55,AL,TVA,0
1,55,MS,TVA,201106
2,59,LA,MISO,132078
3,84,MD,PJM,2565
4,84,VA,PJM,672045


In [23]:
ba_county.head()

,Data Year,Utility Number,Utility Name,State_x,County,State_y,BA_CODE,Megawatthours.4
0,2016.0,34,City of Abbeville - (SC),SC,Abbeville,NaN,NaN,NaN
1,2016.0,55,City of Aberdeen - (MS),MS,Monroe,AL,TVA,0
2,2016.0,55,City of Aberdeen - (MS),MS,Monroe,MS,TVA,201106
3,2016.0,59,City of Abbeville - (LA),LA,Vermilion,LA,MISO,132078
4,2016.0,84,A & N Electric Coop,MD,Somerset,MD,PJM,2565


In [5]:
ba_county_uniq = ba_county[['BA_CODE','County','State_x','State_y']].drop_duplicates(keep='first')
ba_county_uniq.drop(columns = ['State_y'], inplace=True)
ba_county_uniq.rename(columns = {'State_x':'State'}, inplace=True)
ba_county_uniq.dropna(axis = 0, how = 'any', inplace=True)
ba_county_uniq.drop_duplicates(keep = 'first', inplace=True)

In [17]:
ba_county_uniq.head()

,BA_CODE,County,State
1,TVA,Monroe,MS
3,MISO,Vermilion,LA
4,PJM,Somerset,MD
6,PJM,Accomack,VA
8,PJM,Northampton,VA


In [15]:
county_ba_dict = defaultdict(list)
state_ba_dict = defaultdict(set)
for index,row in ba_county_uniq.iterrows():
    county_state = str(row['County']) + '_' + str(row['State'])
    county_ba_dict[county_state].append(row['BA_CODE'])
    state = str(row['State'])
    state_ba_dict[state].add(row['BA_CODE'])

In [12]:
county_ba_dict['Lincoln_NV']

[]

In [16]:
state_ba_dict

defaultdict(set,
            {'MS': {'MISO', 'SOCO', 'TVA'},
             'LA': {'MISO', 'SWPP'},
             'MD': {'PJM'},
             'VA': {'LGEE', 'PJM', 'TVA'},
             'IL': {'EEI', 'MISO', 'PJM'},
             'WI': {'MISO'},
             'OH': {'OVEC', 'PJM'},
             'NY': {'ISNE', 'NYIS', 'PJM'},
             'GA': {'DUK', 'SEPA', 'SOCO', 'TVA'},
             'MN': {'MISO', 'PACW', 'SWPP', 'WACM', 'WALC'},
             'SC': {'CPLE', 'DUK', 'SC', 'SCEG'},
             'AZ': {'AZPS',
              'PACE',
              'PACW',
              'PNM',
              'SRP',
              'SWPP',
              'TEPC',
              'WACM',
              'WALC'},
             'AL': {'AEC', 'MISO', 'SOCO', 'TVA'},
             'FL': {'AEC',
              'FMPP',
              'FPC',
              'FPL',
              'GVL',
              'HST',
              'JEA',
              'NSB',
              'SEC',
              'SOCO',
              'TAL',
              'TEC'},
  

In [14]:
# for keys, values in county_ba_dict.items():
#     if not values:
#         print (keys)
county_ba_dict

defaultdict(list,
            {'Monroe_MS': ['TVA'],
             'Vermilion_LA': ['MISO'],
             'Somerset_MD': ['PJM'],
             'Accomack_VA': ['PJM'],
             'Northampton_VA': ['PJM'],
             'Adams_IL': ['MISO'],
             'Brown_IL': ['MISO'],
             'Schuyler_IL': ['MISO'],
             'Adams_WI': ['MISO'],
             'Columbia_WI': ['MISO'],
             'Dane_WI': ['MISO'],
             'Dodge_WI': ['MISO'],
             'Green Lake_WI': ['MISO'],
             'Jefferson_WI': ['MISO'],
             'Marquette_WI': ['MISO'],
             'Portage_WI': ['MISO'],
             'Sauk_WI': ['MISO'],
             'Waupaca_WI': ['MISO'],
             'Waushara_WI': ['MISO'],
             'Wood_WI': ['MISO'],
             'Adams_OH': ['PJM'],
             'Brown_OH': ['PJM'],
             'Highland_OH': ['PJM'],
             'Pike_OH': ['PJM', 'OVEC'],
             'Scioto_OH': ['PJM'],
             'Cattaraugus_NY': ['NYIS'],
             'Erie_NY': 

# Find the county that a specific bus belongs to

In [112]:
from powersimdata.input.grid import Grid
# eastern = Grid(['Eastern'])
western = Grid(['Western'])

--> Loading Eastern interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line
--> Loading Western interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


In [113]:
western.bus.head()

,type,Pd,Qd,Gs,Bs,zone_id,Vm,Va,baseKV,Vmax,Vmin,lam_P,lam_Q,mu_Vmax,mu_Vmin,interconnect,lat,lon
bus_id,,,,,,,,,,,,,,,,,,
2010001,1,9.64,2.54,0,0.0,201,1.011472,-38.170877,138.0,1.1,0.9,0.0,0.0,0.0,0.0,Western,48.241400,-124.577777
2010002,1,15.98,6.04,0,0.0,201,1.016888,-35.366581,138.0,1.1,0.9,0.0,0.0,0.0,0.0,Western,47.695555,-124.183645
2010003,1,14.39,1.09,0,0.0,201,1.031361,-32.359590,138.0,1.1,0.9,0.0,0.0,0.0,0.0,Western,47.040041,-124.056969
2010004,1,0.00,0.00,0,0.0,201,1.032374,-32.246548,138.0,1.1,0.9,0.0,0.0,0.0,0.0,Western,46.927528,-124.171950
2010005,1,10.45,2.75,0,0.0,201,1.032323,-32.252838,138.0,1.1,0.9,0.0,0.0,0.0,0.0,Western,46.927528,-124.171950


In [150]:
bus_ba_map = western.bus[western.bus['Pd']>0][['Pd','lat','lon']].copy()
bus_ba_map.loc[:,'BA'] = None
bus_ba_map

,Pd,lat,lon,BA
bus_id,,,,
2010001,9.64,48.241400,-124.577777,None
2010002,15.98,47.695555,-124.183645,None
2010003,14.39,47.040041,-124.056969,None
2010005,10.45,46.927528,-124.171950,None
2010006,13.44,46.316481,-124.061190,None
2010009,16.82,46.498255,-123.982546,None
2010010,33.49,47.174349,-123.847397,None
2010011,32.02,48.002536,-123.762001,None
2010013,57.79,47.164996,-123.663897,None


In [151]:
# importing the requests library 
import requests
from collections import Counter
  
# api-endpoint 
URL = "https://geo.fcc.gov/api/census/block/find"
# defining a params dict for the parameters to be sent to the API
count = 0
county_bus_num_count = Counter()
for index,row in bus_ba_map.iterrows():
    PARAMS = {'latitude':row['lat'], 'longitude':row['lon'], 'format':'json', 'showall': True} 

    # sending get request and saving the response as response object 
    r = requests.get(url = URL, params = PARAMS).json()
    try:
        county_ba_dict[r['County']['name']+'_'+r['State']['code']]
    except Exception as err:
        count += 1
        continue
    county_name = r['County']['name']+'_'+r['State']['code']
    county_bus_num_count[county_name] += 1
    ba_num = len(county_ba_dict[county_name])
    bus_ba_map.at[index,'BA'] = county_ba_dict[county_name][county_bus_num_count[county_name]%ba_num]    

ZeroDivisionError: integer division or modulo by zero

In [152]:
bus_ba_map

,Pd,lat,lon,BA
bus_id,,,,
2010001,9.64,48.241400,-124.577777,BPAT
2010002,15.98,47.695555,-124.183645,BPAT
2010003,14.39,47.040041,-124.056969,BPAT
2010005,10.45,46.927528,-124.171950,BPAT
2010006,13.44,46.316481,-124.061190,BPAT
2010009,16.82,46.498255,-123.982546,BPAT
2010010,33.49,47.174349,-123.847397,BPAT
2010011,32.02,48.002536,-123.762001,BPAT
2010013,57.79,47.164996,-123.663897,BPAT


In [10]:
county_name = 'Lincoln_NV'
county_ba_dict[county_name]
# county_name

[]

In [153]:
count

3

In [146]:
county_bus_num_count

Counter({'Clallam_WA': 1,
         'Jefferson_WA': 1,
         'Grays Harbor_WA': 4,
         'Pacific_WA': 4})

In [148]:
county_ba_dict['Pacific_WA']

['BPAT']